<H1>TP3 Analyse de données<H1>

## Contexte
    
Dans ce TP, nous utiliserons l'exemple du jeu de données du Titanic, un énorme paquebot pour l'époque qui fait naufrage en 1912 à la suite d'une collision avec un iceberg, lors de son voyage inaugural de Southampton à New York.
   * Nous avons accès à des informations sur une partie des passagers (1309 passagers) du Titanic. 
   * Pourquoi certains passagers ont survécu et d'autres sont morts?
   * Commencons l'analyse de données
   
![img](Titanic/titanic.jpeg)


## Chargement des données
Les données sont dans un tableau au format CSV (comma separated values):

In [ ]:
import pandas as pd
import numpy as np

titanic = pd.read_csv("test/data-titanic.csv")

Ensuite, on OBSERVE ce qu’on a!!

In [ ]:
titanic.head(10)

In [ ]:
titanic.shape

In [ ]:
titanic.describe(include='all')
#titanic.info()

Les colonnes sont:

* PassengerId : Passenger Id

* Survived : True (1) / False (0)

* Pclass : Passenger ticket class : Class 1, 2 and 3.

* Name : Name of the passenger

* Sex : Sex of the passenger male/female

* Age : Age in years

* Sibsp :	# of siblings / spouses aboard the Titanic	

* Parch	: # of parents / children aboard the Titanic	

* Ticket :	Ticket number	

* fare :	Passenger fare	

* Cabin : Cabin number

* Embarked : Embarkation port ( C : Cherbourg; Q Queenstown; S Southampton)

## Question / Objectif
<strong>Pourquoi certains passagers ont survécu et d’autres sont morts?<strong>

On veut trouver les colonnes qui expliquent le 0/1 dans Survived et construire un modèle qui permettrait d’expliquer au mieux Survived étant donné nos informations.

On va commencer par observer nos données, en répondant à des questions descriptives:
1. Quel sexe a le plus de chances de survie ?
2. Est-ce que les enfants ont eu plus de chances de survie ?
3. Calculer la proportion de survie selon le port d'embarquement.

### Etape 1: Supprimer les NAN de la colonnes "Survived"
La colonne "Survived" a des valeurs manquantes (891 disponibles/1309). Comme la plupart des passagers et passagères sont mort, on peut supposer que Survived =0.

Attention ceci est un choix. Toujours garder en tête qu’il modifie vos résultats et peut donc modifier vos interpétations ! Ici, c’est vraiment à la marge

In [ ]:
titanic["Survived"] = titanic["Survived"].fillna(0.0)

### Quel sexe a le plus de chances de survie ?
* Comparaison de la proportion d’hommes et de femmes passagers du Titanic qui ont survécu

* Utilisons groupby, qui permet de produire des tables de synthèses par catégories:

In [ ]:
titanic.groupby(['Sex','Survived']).count()['PassengerId']

#### Autres tables de synthèse

In [ ]:
passengers = titanic.groupby('Sex')['PassengerId'].count()
passengers

In [ ]:
survivors = titanic.groupby('Sex')['Survived'].sum()
survivors

In [ ]:
summary = pd.DataFrame({"Survivants": survivors,
                        "Passagers": passengers,
                        "%": round(100*survivors / passengers,1)})
summary

#### Visualisation

In [ ]:
import matplotlib.pyplot as plt
summary[["Survivants", "Passagers"]].plot(kind='bar');

Le même graphique, avec titre et labels:

In [ ]:
summary[["Survivants", "Passagers"]].plot(kind='bar');
plt.xlabel('Sexe')
plt.ylabel('Total')
plt.title('Comparaison de la survie selon le sexe');

#### Conclution
* Observation :

  * 26.1% des passengers ont survécus dont 50% des femmes contre seulement 12.9% des hommes

  * Il y a plus d’hommes que de femmes sur le paquebot

* Interprétation : Les femmes ont eu plus de chances de survivre que les hommes

Pour aller plus loin, on pourrait regarder à quel age les hommes et femmes avaient la plus grande chance de survie.

### Est-ce que les enfants ont eu plus de chances de survie ?
* On va commencer par séparer les enfants des adultes selon l’age. Problème, on a des données manquantes.

#### Gestion des données manquantes
Il manque certaines informations. Que feriez-vous ?

On pourrait décider de supprimer les individus sans informations sur l’âge (pensez à vérifier les dimensions de votre table!)

In [ ]:
print("Si j'enlève toutes les lignes contenant un 'NaN': ", titanic.dropna().shape)
titanic_filt_age = titanic.loc[titanic['Age'].notna(),:]
print("\nSi je n'enlève que les 'NaN' de la colonne Age : ", titanic_filt_age.shape)

Maintenant on va créer une nouvelle colonne indiquant si l’on est adulte

In [ ]:
titanic['Adult'] = titanic['Age'] >=18  
titanic.head()

Et quid des individus dont on ne connait pas l’age ?

In [ ]:
titanic_filt_age['Adult'] = titanic_filt_age['Age'] >=18  
passengers = titanic_filt_age.groupby(['Adult','Sex']).count()['PassengerId']
passengers

In [ ]:
survivors = titanic_filt_age.groupby(['Adult','Sex'])['Survived'].sum()
survivors

#### Résumons et Visualisons

In [ ]:
passengers = titanic_filt_age.groupby(['Adult','Sex'])['PassengerId'].count()
summary = pd.DataFrame({"Survivants": survivors,
                       "Passagers": passengers,
                       "%": round(survivors/passengers*100, 1)})
summary.index=['Girl','Boy','Woman','Man']
summary

In [ ]:
summary.plot(kind='bar')
plt.xlabel("Personnes classées selon l'age et le sexe")
plt.ylabel('Total')
plt.title("Comparaison de la survie selon l'age et le sexe");

* Observations:

 * Il y a plus d’adultes que d’enfants.

 * Ainsi, on a respectivement 52.8%, 28%, 50.3% et 12.2% de survivant-es parmi les filles, garcons, femmes et hommes.

* Interprétation: 
  Quelque soit la catégorie, les personnes de sexe féminin ont une plus grande chance de survie que les masculins. Les enfants de sexe masculin ont une plus grande chance de survie que les adultes mais ce n’est pas réciproque pour les personnes de sexe féminin. Pour aller plus loin, que pourrions nous regarder ?

### Calculer la proportion de survie selon le port d’embarquement

La colonne du port d’embarquement a des valeurs manquantes. Comme la plupart des passagers et passagères sont montées à Southampton, on peut supposer que les données manquantes viennent de là.


In [ ]:
titanic["Embarked"] = titanic["Embarked"].fillna('S')

In [ ]:
survivors_per_port = titanic.groupby('Embarked')['Survived'].sum()
passengers_per_port = titanic.groupby('Embarked')['PassengerId'].count()
comparaison_port_survie = pd.DataFrame({"Survivants": survivors_per_port,
                                        "Passagers": passengers_per_port,
                                        "%": round(survivors_per_port/passengers_per_port*100, 1)})
comparaison_port_survie

In [ ]:
comparaison_port_survie.plot(kind='bar')
plt.xlabel("Port d'Embarquement")
plt.ylabel("Nombre d'individus")
plt.title('Comparaison de survie selon le port')

La figure indique que:

* la plupart des individus sont montés à Southampton puis Cherbourg puis Queenstown.

* le nombre de survivants et survivantes est plus grand selon le meme ordre.

* Respectivement 219/916; 93/270 et 30/123 ont survécus selon le port d’embarquement Southampton, Cherbourg, Queenstown.

* L’analyse de proportionnalité nous informe que les individus étant montés à Cherbourg ont eu plus de chance de survie. Pourquoi cela ?

### C'est quoi la corrélation ? Et la causalité ?
Il existe trois types de relations statistiques:

* corrélation positive: si une variable augmente, l’autre aussi.

* corrélation négative: si une variable augmente, l’autre diminue.

* absence de corrélation: si une variable augmente, l’autre peut ou pas varier sans lien entre elle.

Exemple de corrélation:

![img](Titanic/correlationChocolatNobels.png)

Cette corrélation est tirée d’un papier de 2012 par F. Messerli.

C. Pissarides, prix Nobel d’économie en 2010 suite à ce papier à commenté:

« To win a Nobel Prize you have to produce something that others haven’t thought about - chocolate that makes you feel good might contribute a little bit. Of course it’s not the main factor but… anything that contributes to a better life and a better outlook in your life then contributes to the quality of your work. »

1. Décrivez la figure

2. Qu’est-ce qu’on observe?

3. Qu’en conclut-on?

Depuis, il a été montré que la corrélation positive est due à la richesse économique (economic wealth).

Alors qu’est ce que la causalité ?

Dans notre exemple, la richesse économique implique

* une plus grande dépense en recherche

* ce qui implique une corrélation positive avec le nombre de prix Nobel.

Par ailleurs et indépendamment, la richesse économique implique :

* de plus grandes dépenses dans les produits de luxe, dont le chocolat.

Causalité n’est pas corrélation
* Les corrélations relèvent de l'observation;

* Les causalités relèvent de l'interprétation !

Retour au port d’embarquement. Pourquoi on survit plus si on a embarqué a Cherbourg ?

Hypothèse 1: Il y a plus de femmes à Cherbourg (?)

Hypothèse 2: On est plus riche à Cherbourg et plus on est riche plus on a survécu (?)



#### Hypothèse 1: Il y a plus de femmes à Cherbourg

In [ ]:
female_per_port = titanic[titanic['Sex']=='female'].groupby('Embarked')['PassengerId'].count()
male_per_port = titanic[titanic['Sex']=='male'].groupby('Embarked')['PassengerId'].count()
pd.DataFrame({"Female": female_per_port,
              "Male" : male_per_port,
              "Total": passengers_per_port,
              "% Female": female_per_port / passengers_per_port
              })

Il n’y a pas plus d’individus féminins à Cherbourg qu’à Queenstown.

#### Hypothèse 2: on est plus riche à Cherbourg et plus on est riche plus on a survécu (?)

In [ ]:
survivors_per_class = titanic.groupby('Pclass')['Survived'].sum()
passengers_per_class = titanic['Pclass'].value_counts()
pd.DataFrame({"Survivants": survivors_per_class,
              "Passagers": passengers_per_class,
              "%": round(survivors_per_class/passengers_per_class*100, 1)})

Il y a une corrélation entre la classe et la probabilité de survie!

On peut même l’expliquer causalement !

Regardons la répartitions entre classes, selon le port d’embarquement

In [ ]:
pclass1_per_port = titanic[titanic['Pclass']==1].groupby('Embarked').count()['PassengerId']
pclass2_per_port = titanic[titanic['Pclass']==2].groupby('Embarked').count()['PassengerId']
pclass3_per_port = titanic[titanic['Pclass']==3].groupby('Embarked').count()['PassengerId']


pd.DataFrame({'Classe 1': pclass1_per_port,
              'Classe 2': pclass3_per_port,
              'Classe 3': pclass3_per_port,
              'Passengers': passengers_per_port,
              '% Classe 1': round(pclass1_per_port/passengers_per_port*100,1)})

#### Observations

* Les passagers ayant embarqué à Cherbourg regroupent principalement des individus de première classe.

* Les passagers qui embarquent à Queenstown (Irlande) sont principalement de la classe 2 et 3 des migrants en route vers les États-Unis.

#### Conclusions

* Les passagers ayant embarqué à Cherbourg arrivent de Paris (France) et sont plutôt riches.

* Il semble que la classe plus que le port d’embarquement a une relation de causalité avec la survie (à vérifier).

### Corrélation (point mathématique)
Le coefficient de [corrélation linéaire de Pearson](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) se calcule facilement en python . Il correspond à la version normalisée par la standard deviation (écart-type) de la covariance. 
Mathematiquement, on a:
$$
\rho_{xy} = \frac{\sigma_{xy}}{\sigma_x\sigma_y}
$$

varie entre -1 et 1 et représente la force de la relation linéaire qui existe entre les 2 vecteurs/séries.

 * 0 : pas de corrélation,
 * 1 : corrélation positive parfaite (si on connait x alors on peut déduire y, les points sont alignés le long d'une droite)
 * -1: corrélation négative parfaite (idem)
 * en réalité, on a souvent des corrélations intermédiaires
« The intention of this contribution was to show that the correlation between chocolate consumption per capita and the number of Nobel laureates per capita (as reported by Messerli, 2012) will vanish if one controls for relevant other variables and if one uses a sophisticated estimation technique. » par Prinz A. L. (2020)
![img](Titanic/correlationsChocolat.jpeg)
On peut calculer la matrice de corrélation qui correspond à la corrélation entre les colonnes d’une table, et utiliser une carte de chaleur (heatmap en anglais) pour mieux la visualiser:

In [ ]:
titanic.corr()
titanic.corr().style.background_gradient(cmap='coolwarm', axis=1)
#remarque on ne peut pas calculer de correlation linéaire 
#avec des données ayant plus de 2 catégories comme le port d'embarquement
# il faudrait faire une ANOVA (off-topic)

Attention: par défaut, pour attribuer des couleurs aux nombres dans une carte de chaleur, Pandas applique une standardisation par colonne. Le "axis=None" assure que la normalisation est appliquée à l’ensemble des valeurs de la table.

Variante, avec "Seaborn":

In [ ]:
import seaborn as sns

sns.heatmap(titanic.corr(), fmt='0.2f', annot=True, square=True);